In [4]:
from datetime import datetime, timedelta
import calendar
from riak import RiakClient
client = RiakClient()

read_table = "WaterMeterData"
write_table = "WaterMeterMonthlyRollUp"

query = """\
CREATE TABLE WaterMeterMonthlyRollUp (
    customer_id       varchar   not null,
    meter_id          varchar   not null,
    reading_date      timestamp not null,
    billing_month     sint64    not null,
    billing_year      sint64    not null,
    total_gallons     double    not null,
    amount_billed     double    not null,
    PRIMARY KEY(
        (customer_id, meter_id, quantum(reading_date, 3000, 'd')),
         customer_id, meter_id, reading_date
    )
)
"""

try:
    client.ts_query(write_table, query)
    print("Table '{}' created successfully".format(table))
except Exception as e:
    print(e)

'Failed to create table WaterMeterMonthlyRollUp: already_active'


In [5]:
try:    
    description = client.table(write_table).describe()
    for column_desc in description.rows:
        print(column_desc)
except Exception as e:
    print(e)

['customer_id', 'varchar', False, 1L, 1L]
['meter_id', 'varchar', False, 2L, 2L]
['reading_date', 'timestamp', False, 3L, 3L]
['billing_month', 'sint64', False, None, None]
['billing_year', 'sint64', False, None, None]
['total_gallons', 'double', False, None, None]
['amount_billed', 'double', False, None, None]


In [6]:
def convert_to_epoch ( date_to_convert ):
    return calendar.timegm(datetime.timetuple( date_to_convert )) * 1000

start_month = 1
end_month = 2
end_year = 2016

data_set = []

while start_month < 13:
    
    start_date = datetime(2016, start_month, 1, 0, 00)
    end_date = datetime(end_year, end_month, 1, 0, 00)
    
    query = """\
        SELECT
            max(total_gallons) - min(total_gallons)
        FROM WaterMeterData
        WHERE time_stamp >= {} AND time_stamp < {} AND
            customer_id = 'CUSTOMER-0001' AND meter_id = 'METER-0001'
    """.format( convert_to_epoch( start_date ), convert_to_epoch( end_date ) )
    
    result_set = client.ts_query(read_table, query)
    
    new_row = ['CUSTOMER-0001','METER-0001', end_date, end_date.month, end_date.year, 
               result_set.rows[0][0], result_set.rows[0][0] * 0.06]
    print new_row
    data_set.append(new_row)
    
    start_month += 1
    if end_month <> 12:
        end_month += 1
    else:
        end_month = 1
        end_year = 2017

result_message = ""
try:   
    table_object = client.table(write_table).new(data_set)
    result = table_object.store()
    result_message = "Records written: {}".format(result)
except Exception as e:
    result_message = "Error: {}".format(e)
print result_message

['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 2, 1, 0, 0), 2, 2016, 3756.0, 225.35999999999999]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 3, 1, 0, 0), 3, 2016, 3552.5, 213.15]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 4, 1, 0, 0), 4, 2016, 3798.5, 227.91]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 5, 1, 0, 0), 5, 2016, 3657.0, 219.42]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 6, 1, 0, 0), 6, 2016, 3758.5, 225.51]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 7, 1, 0, 0), 7, 2016, 4995.5, 299.72999999999996]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 8, 1, 0, 0), 8, 2016, 5762.5, 345.75]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 9, 1, 0, 0), 9, 2016, 6128.5, 367.71]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 10, 1, 0, 0), 10, 2016, 3675.5, 220.53]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 11, 1, 0, 0), 11, 2016, 3780.0, 226.79999999999998]
['CUSTOMER-0001', 'M

In [7]:
start_date = datetime(2016, 1, 1, 0, 00)
end_date = datetime(2017, 2, 1, 0, 00)

query = """\
    SELECT *
    FROM WaterMeterMonthlyRollUp
    WHERE reading_date >= {} AND reading_date < {} AND
        customer_id = 'CUSTOMER-0001' AND meter_id = 'METER-0001'
""".format( convert_to_epoch( start_date ), convert_to_epoch( end_date ) )

data_set = client.ts_query(write_table, query)
for row in data_set.rows:
    print(row)

['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 2, 1, 0, 0), 2L, 2016L, 3756.0, 225.35999999999999]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 3, 1, 0, 0), 3L, 2016L, 3552.5, 213.15]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 4, 1, 0, 0), 4L, 2016L, 3798.5, 227.91]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 5, 1, 0, 0), 5L, 2016L, 3657.0, 219.42]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 6, 1, 0, 0), 6L, 2016L, 3758.5, 225.51]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 7, 1, 0, 0), 7L, 2016L, 4995.5, 299.72999999999996]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 8, 1, 0, 0), 8L, 2016L, 5762.5, 345.75]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 9, 1, 0, 0), 9L, 2016L, 6128.5, 367.71]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 10, 1, 0, 0), 10L, 2016L, 3675.5, 220.53]
['CUSTOMER-0001', 'METER-0001', datetime.datetime(2016, 11, 1, 0, 0), 11L, 2016L, 3780.0, 226.79999999999998]
